### Field report: PNL and hit rate analysis for 5 basic strategies.
### 17/03/17- v0.1
#### por mrg

In [130]:
#Load packages
Pkg.add("TimeSeries");Pkg.add("MarketData");Pkg.add("Plots");Pkg.add("PlotlyJS");
Pkg.add("Gadfly");Pkg.add("StatsBase");Pkg.update();

INFO: Nothing to be done
INFO: Nothing to be done
INFO: Nothing to be done
INFO: Nothing to be done
INFO: Nothing to be done
INFO: No packages to install, update or remove
INFO: Package database updated
INFO: Updating METADATA...
INFO: Updating cache of DataFrames...
INFO: Updating cache of DataFrames...
INFO: Updating OhMyREPL master...
INFO: Updating Atom master...
INFO: Updating Tokenize master...
INFO: Computing changes...
INFO: No packages to install, update or remove


LoadError: Update finished with errors.
=> Package DataFrames: unable to update cache.
GitError(Code:ERROR, Class:OS, Failed to receive response: The server returned an invalid or unrecognized response
)
 in macro expansion at .\libgit2\error.jl:99 [inlined]
 in #fetch#55(::Base.LibGit2.FetchOptions, ::String, ::Function, ::Base.LibGit2.GitRemote, ::Array{AbstractString,1}) at .\libgit2\remote.jl:70
 in (::Base.LibGit2.#kw##fetch)(::Array{Any,1}, ::Base.LibGit2.#fetch, ::Base.LibGit2.GitRemote, ::Array{AbstractString,1}) at .\<missing>:0
 in #fetch#96(::String, ::String, ::Array{AbstractString,1}, ::Nullable{Base.LibGit2.AbstractCredentials}, ::Function, ::Base.LibGit2.GitRepo) at .\libgit2\libgit2.jl:164
 in fetch(::Base.LibGit2.GitRepo) at .\libgit2\libgit2.jl:156
 in prefetch(::String, ::String, ::Array{String,1}) at .\pkg\cache.jl:64
 in update(::String, ::Set{String}) at .\pkg\entry.jl:403
 in (::Base.Pkg.Dir.##2#3{Array{Any,1},Base.Pkg.Entry.#update,Tuple{String,Set{String}}})()

In [98]:
using TimeSeries, MarketData, Plots, PlotlyJS, StatsBase;

INFO: Recompiling stale cache file C:\Users\miguel\.julia\lib\v0.5\StatsBase.ji for module StatsBase.


In [131]:
typeof(ohlc)

TimeSeries.TimeArray{Float64,2,Date,Array{Float64,2}}

In [132]:
ohlc

500x4 TimeSeries.TimeArray{Float64,2,Date,Array{Float64,2}} 2000-01-03 to 2001-12-31

             Open      High      Low       Close     
2000-01-03 | 104.88    112.5     101.69    111.94    
2000-01-04 | 108.25    110.62    101.19    102.5     
2000-01-05 | 103.75    110.56    103.0     104.0     
2000-01-06 | 106.12    107.0     95.0      95.0      
⋮
2001-12-26 | 21.35     22.3      21.14     21.49     
2001-12-27 | 21.58     22.25     21.58     22.07     
2001-12-28 | 21.97     23.0      21.96     22.43     
2001-12-31 | 22.51     22.66     21.83     21.9      


In [133]:
ohlc["Open"]

500x1 TimeSeries.TimeArray{Float64,1,Date,Array{Float64,1}} 2000-01-03 to 2001-12-31

             Open      
2000-01-03 | 104.88    
2000-01-04 | 108.25    
2000-01-05 | 103.75    
2000-01-06 | 106.12    
⋮
2001-12-26 | 21.35     
2001-12-27 | 21.58     
2001-12-28 | 21.97     
2001-12-31 | 22.51     


In [134]:
import TimeSeries.moving

In [135]:
# Función ventana
function moving{T}(ta::TimeArray{T,1}, f::Function, window::Int; padding::Bool=false)
    tstamps = padding ? ta.timestamp : ta.timestamp[window:end]
    vals    = zeros(ta.values[window:end])
    @inbounds for i=1:length(vals)
        vals[i] = f(ta.values[i:i+(window-1)])
    end
    padding && (vals = [NaN*ones(window-1); vals])
    TimeArray(tstamps, vals, ta.colnames, ta.meta)
end

function moving{T}(ta::TimeArray{T,2}, f::Function, window::Int; padding::Bool=false)
    tstamps = padding ? ta.timestamp : ta.timestamp[window:end]
    vals    = zeros(ta.values[window:end, :])
    @inbounds for i=1:size(vals,1), j=1:size(vals, 2)
        vals[i, j] = f(ta.values[i:i+(window-1), j])
    end
    padding && (vals = [NaN*ones(ta.values[1:(window-1), :]); vals])
    TimeArray(tstamps, vals, ta.colnames, ta.meta)
end

moving (generic function with 2 methods)

In [136]:
#Nice, ya podemos tomar promedios en ventanas de tamaño N = 10
moving(ohlc,mean,10)

D<:Base.Dates.TimeType, A<:AbstractArray}, Function, Int64) in module Main at In[128]:3 overwritten at In[135]:3.

491x4 TimeSeries.TimeArray{Float64,2,Date,Array{Float64,2}} 2000-01-14 to 2001-12-31

             Open      High      Low       Close     
2000-01-14 | 100.692   103.981   96.001    98.782    
2000-01-18 | 100.304   103.331   95.876    97.982    
2000-01-19 | 100.041   103.144   96.095    98.388    
2000-01-20 | 101.216   104.238   97.145    99.338    
⋮
2001-12-26 | 21.062    21.507    20.618    21.065    
2001-12-27 | 21.033    21.54     20.651    21.123    
2001-12-28 | 21.081    21.685    20.797    21.266    
2001-12-31 | 21.259    21.868    20.971    21.417    


In [137]:
fs = (:mean)

for f in fs
@eval ($f) = moving(ohlc,$f,10)
end

Array{Any, 1}, TimeSeries.#moving, TimeSeries.TimeArray{#T<:Any, 1, D<:

LoadError: MethodError: no method matching start(::Symbol)[0m
Closest candidates are:
  start([1m[31m::SimpleVector[0m) at essentials.jl:170
  start([1m[31m::Base.MethodList[0m) at reflection.jl:258
  start([1m[31m::IntSet[0m) at intset.jl:184
  ...[0m

In [139]:
# Aquí se van a desarrollar las funciones estrategia